In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
# from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add
from tensorflow.keras.optimizers import Adam, SGD, Nadam
# from tensorflow.keras import regularizers
import h5py
import numpy as np

In [2]:
tf.__version__
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
def dataGenerator(n_samples=0, start_index=-1, stop_index=-1):
    def generator():
        if (start_index>0 and stop_index>0):
            indexes = np.arange(start_index, stop_index)
        else:
            indexes = np.arange(n_samples)
            
        for i in indexes:
            x = np.array(hdf5['xs'][i])
            
            last_x = -1
            last_y = -1
            for j in range(0, 19):
                for k in range(0, 19):
                    if x[9][j][k] == 1:
                        last_x = j
                        last_y = k
            
            rad = 3
            add_fm = np.zeros((2, 19, 19))
            for j in range(max(0, last_x - rad), min(19, last_x + rad + 1)):
                for k in range(max(0, last_y - rad), min(19, last_y + rad + 1)):
                    for m in range(0, 4, 1):
                        add_fm[0][j][k] += x[m][j][k]
                        add_fm[1][j][k] += x[m + 4][j][k]
            x = np.concatenate((x, add_fm), axis = 0)
#             x.append(add_fm[1])
            x = np.moveaxis(x,0,-1)
#             print(np.shape(x))
#             y = np.array(hdf5['ys'][i])
#             arr = []
#             arr.append()
#             y = np.array([game_styles[i] - 1])
            y = (tf.one_hot(game_styles[i] - 1, depth = 3)).numpy()
#             print(y)
            #印出棋盤
#             print('sample ', end = "")
#             print(i)
#             for i in range(0, 15, 1):
#                 for j in range(0, 19, 1):
#                     for k in range(0, 19, 1):
#                         print(int(x[j][k][i]), end = " ")
#                     print()
#                 print()
                            
#             print(game_styles[i])

#             for i in range(9, 10, 1):
#                 for j in range(0, 19, 1):
#                     for k in range(0, 19, 1):
#                         if x[j][k][i] == 1:
#                             print(j, k)
                            
#             print(y)
#             print(np.shape(y))
#             print(i)
            yield x, y
    return generator

In [4]:
hdf5 = h5py.File('./hdf5/style_dataset_detlef.hdf5','r')
# for j in range (0, 13):
#     print(hdf5['xs'][50][j])
# f = hdf5.get('xs')
# cnt = 0
# for i in range(9, 13, 1):
#     for j in range(0, 19, 1):
#         for k in range(0, 19, 1):
#             if f[1][i][j][k] == 1:
#                 print(i, j, k)
#                 cnt += 1
# print(cnt)
df = open('./Training Dataset/play_style_train.csv').read().splitlines()
game_styles = [int(i.split(',',2)[-2]) for i in df]

batch_size = 32
dataset_size = hdf5['xs'].shape[0]
# val_split = 0.01
train_steps = int(dataset_size / batch_size)
# val_steps = int(dataset_size * val_split / batch_size)


data_gen = dataGenerator(n_samples=int(dataset_size)) # 90% of the complete dataset
# data_gen = dataGenerator(start_index=int(dataset_size-1), # 10% of the complete dataset
#                                stop_index=dataset_size)
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.float32),
                                         output_shapes=(tf.TensorShape((19,19,15)),tf.TensorShape((3))))
dataset = dataset.batch(batch_size, drop_remainder=False)
dataset = dataset.prefetch(tf.data.AUTOTUNE)
# data_gen_valid = dataGenerator(
#                                 start_index=int(dataset_size * (1 - val_split)), # 10% of the complete dataset
#                                stop_index=dataset_size, 
#                                shuffle=False)
# dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
#                                          output_types=(tf.dtypes.float32, tf.dtypes.int32),
#                                          output_shapes=(tf.TensorShape((19,19,13)),tf.TensorShape((1))))
# dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
# dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


# Training

### Simple DCNN Model:

In [5]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # 這邊就是圖5上的1x1x64降維操作，假設input x的維度是(n, n, 256), channel last
    x = layers.Conv2D(filters1, (1, 1),
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)
    
    # 正常的3x3x64卷積操作，Feature Map長寬仍是n x n
    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)
    
    # 最後升維到256，維度(n,n,256) -> 變成可以和(Indentity)input x相加的維度
    x = layers.Conv2D(filters3, (1, 1),
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # 相加後做non-linear轉換
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

In [6]:
def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(1, 1)):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 因為是projection shortcut 所以input的x可能跟output維度不同
    # input維度(n,n,256) -->降維 (n,n,64)
    # 如果Strides有改，則利用Strides來改變Feature Map長寬
    x = layers.Conv2D(filters1, (1, 1), strides=strides,  
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)
    
    # (3,3)的kernel, padding都pad好pad滿，不改變Feature Map尺寸大小
    x = layers.Conv2D(filters2, kernel_size, padding='same', 
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)
    
    # 用1x1 conv升維到假設512
    x = layers.Conv2D(filters3, (1, 1), 
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # 因input維度是256，這邊就需要做projectr將維度升到512相加
    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
#                              kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)
    # F(x) + x(升維後的x)
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [7]:
def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    img_input = layers.Input(shape = input_shape)
    bn_axis = 3 #unknow
    ##### optional
#     x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
#     x = layers.Conv2D(32, (7, 7),
#                       strides=(2, 2),
#                       padding='same',
#                       kernel_initializer='he_normal',
#                       name='conv1')(img_input)
#     x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
#     x = layers.Activation('relu')(x)
    #####
    
    x = conv_block(img_input, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))  # input Channel大小會跟最後最後residual output尺寸一樣
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    # 256-d to 512-d
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a') # projection shortcut
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    # 512-d to 1024-d
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a') # projection shortcut
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
#     # 1024-d to 2048-d
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(3, activation='softmax', name='fc1000')(x)
#     if include_top:
#         x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
#         x = layers.Dense(classes, activation='softmax', name='fc1000')(x)
#     else:
#         if pooling == 'avg':
#             x = layers.GlobalAveragePooling2D()(x)
#         elif pooling == 'max':
#             x = layers.GlobalMaxPooling2D()(x)
#         else:
#             warnings.warn('The output shape of `ResNet50(include_top=False)` '
#                           'has been changed since Keras 2.2.0.')
#     x = layers.GlobalAveragePooling2D()(x)
    
        
    model = Model(img_input, x, name='resnet50')
    return model

In [10]:
model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 15),
                 pooling=False)
opt = Nadam(learning_rate = 0.000001)
model.compile(optimizer = opt,
              loss='categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 19, 19, 15)  0           []                               
                                ]                                                                 
                                                                                                  
 res2a_branch2a (Conv2D)        (None, 19, 19, 64)   1024        ['input_2[0][0]']                
                                                                                                  
 bn2a_branch2a (BatchNormalizat  (None, 19, 19, 64)  256         ['res2a_branch2a[0][0]']         
 ion)                                                                                             
                                                                                           

 res3a_branch2a (Conv2D)        (None, 19, 19, 128)  32896       ['activation_56[0][0]']          
                                                                                                  
 bn3a_branch2a (BatchNormalizat  (None, 19, 19, 128)  512        ['res3a_branch2a[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_57 (Activation)     (None, 19, 19, 128)  0           ['bn3a_branch2a[0][0]']          
                                                                                                  
 res3a_branch2b (Conv2D)        (None, 19, 19, 128)  147584      ['activation_57[0][0]']          
                                                                                                  
 bn3a_branch2b (BatchNormalizat  (None, 19, 19, 128)  512        ['res3a_branch2b[0][0]']         
 ion)     

                                                                                                  
 res3d_branch2b (Conv2D)        (None, 19, 19, 128)  147584      ['activation_66[0][0]']          
                                                                                                  
 bn3d_branch2b (BatchNormalizat  (None, 19, 19, 128)  512        ['res3d_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_67 (Activation)     (None, 19, 19, 128)  0           ['bn3d_branch2b[0][0]']          
                                                                                                  
 res3d_branch2c (Conv2D)        (None, 19, 19, 512)  66048       ['activation_67[0][0]']          
                                                                                                  
 bn3d_bran

 res4c_branch2b (Conv2D)        (None, 19, 19, 256)  590080      ['activation_75[0][0]']          
                                                                                                  
 bn4c_branch2b (BatchNormalizat  (None, 19, 19, 256)  1024       ['res4c_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_76 (Activation)     (None, 19, 19, 256)  0           ['bn4c_branch2b[0][0]']          
                                                                                                  
 res4c_branch2c (Conv2D)        (None, 19, 19, 1024  263168      ['activation_76[0][0]']          
                                )                                                                 
                                                                                                  
 bn4c_bran

 activation_85 (Activation)     (None, 19, 19, 256)  0           ['bn4f_branch2b[0][0]']          
                                                                                                  
 res4f_branch2c (Conv2D)        (None, 19, 19, 1024  263168      ['activation_85[0][0]']          
                                )                                                                 
                                                                                                  
 bn4f_branch2c (BatchNormalizat  (None, 19, 19, 1024  4096       ['res4f_branch2c[0][0]']         
 ion)                           )                                                                 
                                                                                                  
 add_28 (Add)                   (None, 19, 19, 1024  0           ['bn4f_branch2c[0][0]',          
                                )                                 'activation_83[0][0]']          
          

                                                                                                  
 activation_94 (Activation)     (None, 19, 19, 512)  0           ['bn5c_branch2b[0][0]']          
                                                                                                  
 res5c_branch2c (Conv2D)        (None, 19, 19, 2048  1050624     ['activation_94[0][0]']          
                                )                                                                 
                                                                                                  
 bn5c_branch2c (BatchNormalizat  (None, 19, 19, 2048  8192       ['res5c_branch2c[0][0]']         
 ion)                           )                                                                 
                                                                                                  
 add_31 (Add)                   (None, 19, 19, 2048  0           ['bn5c_branch2c[0][0]',          
          

In [39]:
def residual_block(x, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def go_res():
    inputs = Input(shape=(19, 19, 15))
    conv5x5 = Conv2D(kernel_size=5,
                     filters=256,
                     padding="same",
                     name='conv5x5')(inputs)
    conv1x1 = Conv2D(kernel_size=1,
                     filters=256,
                     padding="same",
                     name='conv1x1')(inputs)
    outputs = Add()([conv5x5, conv1x1])
    outputs = ReLU()(outputs)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
#     outputs = Conv2D(kernel_size=3,
#                      filters=1,
#                      padding="same")(outputs)
#     outputs = ReLU()(outputs)
#     outputs = Flatten()(outputs)
#     outputs = Softmax()(outputs)
    outputs = GlobalAveragePooling2D(name='avg_pool')(outputs)
    outputs = Dense(3, activation='softmax', name='fc1000')(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    return model

model = go_res()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 19, 19, 15)  0           []                               
                                ]                                                                 
                                                                                                  
 conv5x5 (Conv2D)               (None, 19, 19, 256)  96256       ['input_8[0][0]']                
                                                                                                  
 conv1x1 (Conv2D)               (None, 19, 19, 256)  4096        ['input_8[0][0]']                
                                                                                                  
 add_76 (Add)                   (None, 19, 19, 256)  0           ['conv5x5[0][0]',          

                                                                                                  
 fc1000 (Dense)                 (None, 3)            771         ['avg_pool[0][0]']               
                                                                                                  
Total params: 7,182,083
Trainable params: 7,182,083
Non-trainable params: 0
__________________________________________________________________________________________________


In [23]:
# model = load_model('./models/model_playstyle_resnet50_v2.h5')

In [11]:
history = model.fit(
    dataset,
    epochs = 1,
#     steps_per_epoch = train_step
#     validation_data=(x_val, y_val),
)

    247/Unknown - 93s 328ms/step - loss: 1.2516 - accuracy: 0.4338

KeyboardInterrupt: 

In [15]:
# model.save('./models/model_playstyle_resnet50_alltrain_withpp.h5')

In [5]:
model = load_model('./models/model_playstyle_resnet50_alltrain_withpp.h5')

In [6]:
history = model.fit(
    dataset,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

Epoch 1/6
    542/Unknown - 127s 202ms/step - loss: 0.7313 - accuracy: 0.6791

KeyboardInterrupt: 

In [20]:
model.save('./models/model_playstyle_resnet50_alltrain_1.h5')

In [21]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

Epoch 1/6
832/832 [==============================] - 535s 643ms/step - loss: 0.4887 - accuracy: 0.8066
Epoch 2/6
832/832 [==============================] - 535s 643ms/step - loss: 0.4530 - accuracy: 0.8203
Epoch 3/6
832/832 [==============================] - 535s 643ms/step - loss: 0.4121 - accuracy: 0.8367
Epoch 4/6
832/832 [==============================] - 535s 643ms/step - loss: 0.3823 - accuracy: 0.8463
Epoch 5/6
832/832 [==============================] - 535s 643ms/step - loss: 0.3455 - accuracy: 0.8637
Epoch 6/6
832/832 [==============================] - 535s 643ms/step - loss: 0.3112 - accuracy: 0.8769


In [22]:
model.save('./models/model_playstyle_resnet50_alltrain_2.h5')

In [13]:
model = load_model('./models/model_playstyle_resnet50_alltrain_2.h5')

In [14]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

Epoch 1/6
832/832 [==============================] - 546s 643ms/step - loss: 0.2820 - accuracy: 0.8899
Epoch 2/6
832/832 [==============================] - 535s 643ms/step - loss: 0.2519 - accuracy: 0.9023
Epoch 3/6
832/832 [==============================] - 535s 642ms/step - loss: 0.2362 - accuracy: 0.9075
Epoch 4/6
832/832 [==============================] - 535s 643ms/step - loss: 0.2164 - accuracy: 0.9179
Epoch 5/6
832/832 [==============================] - 534s 642ms/step - loss: 0.1968 - accuracy: 0.9235
Epoch 6/6
832/832 [==============================] - 534s 642ms/step - loss: 0.1818 - accuracy: 0.9309


In [15]:
model.save('./models/model_playstyle_resnet50_alltrain_3.h5')

In [12]:
model = load_model('./models/model_playstyle_resnet50_alltrain_3.h5')

In [13]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

Epoch 1/6
832/832 [==============================] - 544s 641ms/step - loss: 0.1721 - accuracy: 0.9356
Epoch 2/6
832/832 [==============================] - 532s 640ms/step - loss: 0.1637 - accuracy: 0.9386
Epoch 3/6
832/832 [==============================] - 533s 640ms/step - loss: 0.1506 - accuracy: 0.9446
Epoch 4/6
832/832 [==============================] - 533s 641ms/step - loss: 0.1439 - accuracy: 0.9459
Epoch 5/6
832/832 [==============================] - 534s 642ms/step - loss: 0.1365 - accuracy: 0.9506
Epoch 6/6
832/832 [==============================] - 534s 642ms/step - loss: 0.1311 - accuracy: 0.9516


In [14]:
model.save('./models/model_playstyle_resnet50_alltrain_4.h5')

In [13]:
model = load_model('./models/model_playstyle_resnet50_alltrain_4.h5')

In [14]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

Epoch 1/6
832/832 [==============================] - 546s 643ms/step - loss: 0.1324 - accuracy: 0.9520
Epoch 2/6
832/832 [==============================] - 535s 643ms/step - loss: 0.1137 - accuracy: 0.9585
Epoch 3/6
832/832 [==============================] - 535s 643ms/step - loss: 0.1263 - accuracy: 0.9528
Epoch 4/6
832/832 [==============================] - 535s 643ms/step - loss: 0.1134 - accuracy: 0.9599
Epoch 5/6
832/832 [==============================] - 535s 643ms/step - loss: 0.1135 - accuracy: 0.9594
Epoch 6/6
832/832 [==============================] - 534s 642ms/step - loss: 0.0914 - accuracy: 0.9671


In [15]:
model.save('./models/model_playstyle_resnet50_alltrain_5.h5')

In [ ]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 6,
#     validation_data=(x_val, y_val),
)

In [ ]:
model.save('./models/model_playstyle_resnet50_alltrain_6.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!